In [1]:
import glob
import pandas as pd
from datetime import datetime as dt
import numpy as np
from datetime import timedelta
from joblib import Parallel, delayed
from tqdm import tqdm

In [2]:
def simple_read(path):
    '''
    Reads .ict files to a Pandas DataFrame
    :param path: path to the .ict data
    :return: Pandas DataFrame with .ict data
    '''
    with open(path) as f:
        # find the value in the file which tells you how many lines to skip to get to the table
        first_line = f.readline()
        header_line = int(first_line[0:-2].split(",")[0])-1
    data = pd.read_csv(path, sep=',', skiprows=header_line)

    # finds the location in the path containing the date
    acc = 0
    boo = False
    for letter in path:
        if letter == '2':
            boo = True
        elif boo and letter == '0':
            acc -= 1
            break
        acc += 1
        
    # creates datetime object with the date the data was collected
    day = dt(int(path[acc:acc+4]), int(path[acc+4:acc+6]), int(path[acc+6:acc+8])) 
    
    for column in data.keys():
        if 'Time' in column:
            # converts seconds after midnight columns to datetime
            data[column] = day + pd.to_timedelta(data[column], unit='seconds')
    data.columns = data.columns.str.replace(' ', '')
    return data.replace(-9999, np.nan) # Converts -9999 values to NaN

In [3]:
def read_instr(instr, subset = None):
    paths = sorted(glob.glob('../data/*'+instr+'*'))
    d_list = []
    for i in range(0, len(paths)):
        d_list.append(simple_read(paths[i]))
    d = pd.concat(d_list).reset_index(drop=True)
    if subset:
        d = d.dropna(subset = subset, how='all').reset_index(drop=True)
    return d

In [4]:
RI = read_instr('DASH', ['RI', 'GF'])
AMS = read_instr('AMS')
OPT = read_instr('OPT')
MetNav = read_instr('MetNav')

In [5]:
def merge_data(d1, d2, n_jobs=-1):

    def row_mean(d1_row, d2, vars):
        # Filter d2 based on the time range in d1_row
        sub = d2[(d2['Time_Start'] >= d1_row['Time_Start']) & (d2['Time_Stop'] <= d1_row['Time_Stop'])].mean()
        # Return the index and the computed values
        return sub[vars]
    
    def row_mean_single_time(d1_row, d2, vars, time_key):
        # Filter d2 based on the time range in d1_row
        sub = d2[(d2[time_key] >= d1_row['Time_Start']) & (d2[time_key] < d1_row['Time_Stop'])].mean()
        # Return the index and the computed values
        return sub[vars]


    # Get the list of variables to merge (excluding 'Time' related columns)
    vars = [var for var in d2.keys() if 'Time' not in var]
    
    if 'Time_Start' and 'Time_Stop' in d2.keys():
        # Use joblib to parallelize the processing of each row in d1
        m = Parallel(n_jobs=n_jobs)(
            delayed(row_mean)(d1_row, d2, vars) for _, d1_row in tqdm(d1.iterrows(), total = len(d1))
        )
    else:
        time_key = [var for var in d2.keys() if 'Time' in var][0]
        m = Parallel(n_jobs=n_jobs)(
            delayed(row_mean_single_time)(d1_row, d2, vars, time_key) for _, d1_row in tqdm(d1.iterrows(), total = len(d1))
        )

    m = pd.concat(m, axis=1).T
    out = pd.merge(d1, m, left_index=True, right_index=True)
    return out

In [6]:
d = merge_data(RI, AMS)
d = merge_data(d, OPT)
d = merge_data(d, MetNav)

100%|██████████| 31681/31681 [01:48<00:00, 291.79it/s]


In [8]:
d.to_csv('../tables/d.csv', index=False)